In [1]:
%matplotlib inline
import os
import time
import pickle
import random
import warnings
import itertools
import numpy as np
from sklearn import svm
from sklearn import metrics
import matplotlib.pyplot as plt
from copy import copy, deepcopy
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support


In [2]:
# warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True, linewidth=150, precision=2)

#### Load and Visulize Y with Quantatitve features

In [3]:
name_of_particle = 'JetHTc'

X_train = np.load("matrices/" + name_of_particle +"_train.npy",)
y_train = np.load("matrices/" + name_of_particle +"_y_train.npy",)
X_val = np.load("matrices/" + name_of_particle +"_val.npy",)
y_val = np.load("matrices/" + name_of_particle +"_y_val.npy",)
X_test = np.load("matrices/" + name_of_particle +"_test.npy",)
y_test = np.load("matrices/" + name_of_particle +"_y_test.npy",)
X_train = X_train[:, :-3]
X_val = X_val[:, :-3]
X_test = X_test[:, :-3]
_, V = X_train.shape
K = 2
V

169

In [14]:
def perf_measure(y_true, y_pred):
    
    cnf_matrix = confusion_matrix(y_true, y_pred)
    
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)
    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)
    
    FSCORE = np.divide((2*PPV*TPR), (PPV+TPR))
    
    return PPV, TPR, FSCORE, FNR, FPR, TNR

### demonstrating the created data sets - Without Noise

Let us compute the PCA of each case and plot the first two principle components. Moreover, we plo the image plot of the corresponding entity-to-feature matrix next to it.


# INITIALLY BLANK






In [5]:
set(y_test)

{0.0, 1.0}

### spilitting data into train and test

In [6]:
y_test = [-1 if i ==1 else 1 for i in y_test]
y_train = [-1 if i ==1 else 1 for i in y_train]

In [7]:
set(y_test), set(y_train)

({-1, 1}, {-1, 1})

#### One-Class SVM

nu=0.1, kernel="rbf", gamma=0.1   >>  precision-OCS: 0.00 recall-kmean: 0.00 fscore-kmean: 0.00, 4h 54min 31s

nu=0.01, kernel="rbf", gamma=auto, shrinking=True


In [8]:
nu = [0.01,]  # 0.1, 0.5, 0.9]
gamma = ['scale']
setting_ocs = list(itertools.product(nu, gamma))
setting_ocs

[(0.01, 'scale')]

In [9]:
for i in setting_ocs:
    start = time.time()
    
    print("train setting:", i)
    
    clf_ocs =  svm.OneClassSVM(nu=i[0], kernel="rbf", gamma=i[1], shrinking=True)
    clf_ocs.fit(X_train)
    labels_pred_ocs_train = clf_ocs.predict(X_train)
    labels_pred_ocs = clf_ocs.predict(X_test)

    with open (os.path.join('RealData_computation', "clustering_ocs_Zero" + str(i) +".pickle"), 'wb') as fp:  # Small_Quantatitive
        pickle.dump(clf_ocs, fp)  
    
#     AMI_ocs = metrics.adjusted_mutual_info_score(y_train, labels_pred_ocs_train)
#     NMI_ocs = metrics.normalized_mutual_info_score(y_train, labels_pred_ocs_train)
#     ARI_ocs = metrics.adjusted_rand_score(y_train, labels_pred_ocs_train)
    FSCORE_ocs = precision_recall_fscore_support(y_train, labels_pred_ocs_train, average='weighted')
    
    
    print("precision-OCS:", "%.2f" % FSCORE_ocs[0], "recall:", "%.2f" % FSCORE_ocs[1], "fscore:", "%.2f" % FSCORE_ocs[2])
    print(" ")
    
    
#     AMI_ocs_test = metrics.adjusted_mutual_info_score(y_test, labels_pred_ocs)
#     NMI_ocs_test = metrics.normalized_mutual_info_score(y_test, labels_pred_ocs)
#     ARI_ocs_test = metrics.adjusted_rand_score(y_test, labels_pred_ocs)
    FSCORE_ocs_test = precision_recall_fscore_support(y_test, labels_pred_ocs, average='weighted')
    print("test")
    print("precision-OCS:", "%.2f" % FSCORE_ocs_test[0], "recall:", "%.2f" % FSCORE_ocs_test[1], "fscore:", "%.2f" % FSCORE_ocs_test[2])
    
    end = time.time()
    print(end-start)
    
    PPV, TPR, FSCORE, FNR, FPR, TNR = perf_measure(y_true=y_test, y_pred=labels_pred_ocs)
    
    PPV, TPR, FSCORE, FNR, FPR, TNR
    

train setting: (0.01, 'scale')
precision-OCS: 0.97 recall: 0.98 fscore: 0.98
 
test
precision-OCS: 0.97 recall: 0.98 fscore: 0.98
62.25480532646179


In [10]:
PPV, TPR, FSCORE, FNR, FPR, TNR = perf_measure(y_true=y_test, y_pred=labels_pred_ocs)
    
PPV, TPR, FSCORE, FNR, FPR, TNR

(array([0.06, 0.99]),
 array([0.05, 0.99]),
 array([0.05, 0.99]),
 array([0.95, 0.01]),
 array([0.01, 0.95]),
 array([0.99, 0.05]))

In [11]:
# for i in setting_ocs:
#     print("test", "setting:", i)
    
#     with open (os.path.join('RealData_computation', "clustering_ocs" + str(i) +".pickle"), 'rb') as fp:  # Small_Quantatitive
#         clf_ocs = pickle.load(fp)  
        
    
#     labels_pred_ocs = clf_ocs.predict(X_test)
    
    
#     FSCORE_ocs_test = precision_recall_fscore_support(y_test, labels_pred_ocs, average='weighted')
    
#     print("precision-OCS:", "%.2f" % FSCORE_ocs_test[0], "recall:", "%.2f" % FSCORE_ocs_test[1], "fscore:", "%.2f" % FSCORE_ocs_test[2])
    

array([[   17,   360],
       [  260, 27396]])

In [ ]:
B